## Sample PointNet

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class PCVNet(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(PCVNet, self).__init__()
        self.output_dim = output_dim

        self.conv_1 = nn.Conv1d(input_dim, 64, 1)
        self.conv_2 = nn.Conv1d(64, 128, 1)
        self.conv_3 = nn.Conv1d(128, 1024, 1)

        self.bn_1 = nn.BatchNorm1d(64)
        self.bn_2 = nn.BatchNorm1d(128)
        self.bn_3 = nn.BatchNorm1d(1024)
        self.bn_4 = nn.BatchNorm1d(512)
        self.bn_5 = nn.BatchNorm1d(256)

        self.fc_1 = nn.Linear(1024, 512)
        self.fc_2 = nn.Linear(512, 256)
        self.fc_3 = nn.Linear(256, self.output_dim * self.output_dim)

        self.max_pool = nn.MaxPool1d(2)

    def forward(self, x):
        x = x.transpose(2, 1)
        x = F.relu(self.bn_1(self.conv_1(x)))
        x = F.relu(self.bn_2(self.conv_2(x)))
        x = F.relu(self.bn_3(self.conv_3(x)))
        x = self.max_pool(x)

        x = x.view(-1, 1024)

        x = F.relu(self.bn_4(self.fc_1(x)))
        x = F.relu(self.bn_5(self.fc_2(x)))
        x = self.fc_3(x)

        identity_matrix = torch.eye(self.output_dim)
        if torch.cuda.is_available():
            identity_matrix = identity_matrix.cuda()
        x = x.view(-1, self.output_dim, self.output_dim) + identity_matrix
        return x


class BasePCVNet(nn.Module):
    def __init__(self, point_dimension, return_local_features=False):
        super(BasePCVNet, self).__init__()
        self.return_local_features = return_local_features
        self.input_transform = PCVNet(input_dim=point_dimension, output_dim=point_dimension)
        self.feature_transform = PCVNet(input_dim=64, output_dim=64)

        self.conv_1 = nn.Conv1d(point_dimension, 64, 1)
        self.conv_2 = nn.Conv1d(64, 64, 1)
        self.conv_3 = nn.Conv1d(64, 64, 1)
        self.conv_4 = nn.Conv1d(64, 128, 1)
        self.conv_5 = nn.Conv1d(128, 1024, 1)

        self.bn_1 = nn.BatchNorm1d(64)
        self.bn_2 = nn.BatchNorm1d(64)
        self.bn_3 = nn.BatchNorm1d(64)
        self.bn_4 = nn.BatchNorm1d(128)
        self.bn_5 = nn.BatchNorm1d(1024)

        self.max_pool = nn.MaxPool1d(2)

    def forward(self, x):
        num_points = x.shape[1]

        input_transform = self.input_transform(x)

        x = torch.bmm(x, input_transform)
        x = x.transpose(2, 1)
        x = F.relu(self.bn_1(self.conv_1(x)))
        x = F.relu(self.bn_2(self.conv_2(x)))
        x = x.transpose(2, 1)

        feature_transform = self.feature_transform(x)

        x = torch.bmm(x, feature_transform)
        local_point_features = x

        x = x.transpose(2, 1)
        x = F.relu(self.bn_3(self.conv_3(x)))
        x = F.relu(self.bn_4(self.conv_4(x)))
        x = F.relu(self.bn_5(self.conv_5(x)))
        x = self.max_pool(x)

        x = x.view(-1, 1024)

        if self.return_local_features:
            x = x.view(-1, 1024, 1).repeat(1, 1, num_points)
            return torch.cat([x.transpose(2, 1), local_point_features], 2), feature_transform
        else:
            return x, feature_transform


class ClassificationPCVNet(nn.Module):

    def __init__(self, num_classes=10, dropout=0.3, point_dimension=1024):
        super(ClassificationPCVNet, self).__init__()
        self.base_pointnet = BasePCVNet(return_local_features=False, point_dimension=point_dimension)

        self.fc_1 = nn.Linear(1024, 512)
        self.fc_2 = nn.Linear(512, 256)
        self.fc_3 = nn.Linear(256, num_classes)

        self.bn_1 = nn.BatchNorm1d(512)
        self.bn_2 = nn.BatchNorm1d(256)

        self.dropout_1 = nn.Dropout(dropout)

    def forward(self, x):
        x, feature_transform = self.base_pointnet(x)

        x = F.relu(self.bn_1(self.fc_1(x)))
        x = F.relu(self.bn_2(self.fc_2(x)))
        x = self.dropout_1(x)

        return F.log_softmax(self.fc_3(x), dim=1), feature_transform


def test():
    model = ClassificationPCVNet(10, 0.3, 3)
    #model = ClassificationPCVNet()
    dummy_input = torch.randn(64, 2, 3)
    x = model(dummy_input)

    torch.save(model.state_dict(), 'pcv_pointnet.pth')
    state_dict = torch.load('pcv_pointnet.pth')
    model.load_state_dict(state_dict)

    dummy_input = torch.randn(64, 2, 3)
    torch.onnx.export(model, dummy_input, 'pcv_pointnet.onnx')





```
# This is formatted as code
```

## Loss Functions

In [3]:
def pointnetloss(outputs, labels, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    #id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        #id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    #diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    #diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64)
    return criterion(outputs, labels) + alpha * torch.norm(diff64x64) / float(bs)

## Download dataset into google colab runtime


In [4]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

--2022-04-13 18:27:59--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.61
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip’

ModelNet10.zip      100%[===================>] 451.47M   102MB/s    in 4.5s    

2022-04-13 18:28:04 (101 MB/s) - ‘ModelNet10.zip’ saved [473402300/473402300]



## Unzip dataset


In [5]:
!unzip -q ModelNet10.zip;

## Install the dependencies on path


In [6]:
!pip install path.py;
from path import Path

In [7]:
path = Path("ModelNet10")

## import Dependencies

In [8]:
import os
import numpy as np

## Check dataset

In [9]:
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

## Data Visualizer

In [10]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [11]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()
    

## One of the important steps: Format dataset.
### This dataset consists of .off files that contain meshes represented by vertices and triangular faces.

### We will need a function to read this type of files:

In [12]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [13]:
with open(path/"bed/train/bed_0001.off", 'r') as f:
  verts, faces = read_off(f)

In [14]:
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [15]:
print(len(x))
print(len(y))
print(len(z))
print(len(i))
print(len(j))
print(len(k))

2095
2095
2095
1807
1807
1807


## Import Dependencies

In [16]:
import math
import random
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

## Sample points

In [17]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    

In [18]:
pointcloud = PointSampler(1024)((verts, faces))

## Normalize

In [19]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [20]:
norm_pointcloud = Normalize()(pointcloud)

## Data Augmentation


In [21]:
class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.03, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

In [22]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)

In [23]:
pcshow(*rot_pointcloud.T)

In [24]:
pcshow(*noisy_rot_pointcloud.T)

## Create Tensors

In [25]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [26]:
ToTensor()(noisy_rot_pointcloud)

tensor([[ 0.4918, -0.7029, -0.0600],
        [-0.0246,  0.4446, -0.0925],
        [ 0.4653,  0.1611, -0.0881],
        ...,
        [-0.0898,  0.1139,  0.1169],
        [-0.2454,  0.4230, -0.0941],
        [ 0.4764,  0.0161,  0.0745]], dtype=torch.float64)

## Create a transformation with PointSampler, normalize and ToTensor

In [27]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

## Let's Create Python custom dataset

In [28]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

## Transform for training. Sample with 1024 points

In [29]:
train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])

In [30]:
train_ds = PointCloudData(path, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', transform=train_transforms)

In [31]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()};
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

## Segment dataset for training and testing

In [32]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())
print('Class: ', inv_classes[train_ds[0]['category']])

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
Sample pointcloud shape:  torch.Size([1024, 3])
Class:  bathtub


In [33]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

## Add GPU execution mode

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Setup Training

In [35]:
#pointnet = PointNet()
pointnet = ClassificationPCVNet()
pointnet.to(device);

In [36]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

In [86]:
def train(model, train_loader, val_loader=None,  epochs=1, save=True):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            #outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
            outputs, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), "save_"+str(epoch)+".pth")
            #torch.onnx.export(model, dummy_input, 'pcv_pointnet.onnx')

In [ ]:
train(pointnet, train_loader, valid_loader,  save=False)

[Epoch: 1, Batch:   10 /  125], loss: 2.212
[Epoch: 1, Batch:   20 /  125], loss: 2.021
[Epoch: 1, Batch:   30 /  125], loss: 1.901


In [37]:
def test():
    #pointnet = ClassificationPCVNet()
    #dummy_input = torch.randn(64, 3, 3)
    count = 1
    for data in valid_loader:
      if count == 2:
        break
      dummy_input = data['pointcloud'].to(device).float()
      x = pointnet(dummy_input.transpose(1,2))
      count = count + 1
    
    

    torch.save(pointnet.state_dict(), 'pcv_pointnet.pth')
    state_dict = torch.load('pcv_pointnet.pth')
    pointnet.load_state_dict(state_dict)

    #dummy_input = torch.randn(64, 3, 3)
    count = 1
    for data in valid_loader:
      if count == 2:
        break
      dummy_input = data['pointcloud'].to(device).float()
      torch.onnx.export(pointnet, dummy_input.transpose(1,2), 'pcv_pointnet.onnx', 
                        export_params=True, opset_version=12, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)
      count = count + 1

    
if __name__ == '__main__':
    test()

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning:

`add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning:

`do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.



In [39]:
pip install onnx onnxruntime

     |████████████████████████████████| 12.8 MB 32.7 MB/s 
     |████████████████████████████████| 5.2 MB 48.8 MB/s 


In [40]:
import onnx

# Load the ONNX model
model = onnx.load("pcv_pointnet.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %0[FLOAT, 64x3x1024]
) optional inputs with matching initializers (
  %base_pointnet.input_transform.bn_4.weight[FLOAT, 512]
  %base_pointnet.input_transform.bn_4.bias[FLOAT, 512]
  %base_pointnet.input_transform.bn_4.running_mean[FLOAT, 512]
  %base_pointnet.input_transform.bn_4.running_var[FLOAT, 512]
  %base_pointnet.input_transform.bn_5.weight[FLOAT, 256]
  %base_pointnet.input_transform.bn_5.bias[FLOAT, 256]
  %base_pointnet.input_transform.bn_5.running_mean[FLOAT, 256]
  %base_pointnet.input_transform.bn_5.running_var[FLOAT, 256]
  %base_pointnet.input_transform.fc_1.weight[FLOAT, 512x1024]
  %base_pointnet.input_transform.fc_1.bias[FLOAT, 512]
  %base_pointnet.input_transform.fc_2.weight[FLOAT, 256x512]
  %base_pointnet.input_transform.fc_2.bias[FLOAT, 256]
  %base_pointnet.input_transform.fc_3.weight[FLOAT, 1048576x256]
  %base_pointnet.input_transform.fc_3.bias[FLOAT, 1048576]
  %base_pointnet.feature_transform.bn_4.weight[FLOAT, 512]
  %base_pointne